In [5]:
import sys
sys.path.insert(1,'/Users/xiaoyuzhao/Dropbox/code/')

In [1]:
%pwd

'/Users/XiaoyuZhao/Dropbox/python notebook/data_parse'

In [6]:
multitags = ["ACAGTGTATATA","ACAGTGCGCTCT","ACAGTGGAGACG","ACAGTGATACTG","ACAGTGACTAGC","ACAGTGTGAGCT","ACAGTGTGCTAC","ACAGTGCGTACG",
             "TGACCATATATA","TGACCACGCTCT","TGACCAGAGACG","TGACCAATACTG","TGACCAACTAGC","TGACCATGAGCT","TGACCATGCTAC","TGACCACGTACG",
             "GCCAATTATATA","GCCAATCGCTCT","GCCAATGAGACG","GCCAATATACTG","GCCAATACTAGC","GCCAATTGAGCT","GCCAATTGCTAC","GCCAATCGTACG",
             "ATCACGTATATA","ATCACGCGCTCT","ATCACGGAGACG","ATCACGATACTG","ATCACGACTAGC","ATCACGTGAGCT","ATCACGTGCTAC","ATCACGCGTACG",
             "GGCTACTATATA","GGCTACCGCTCT","GGCTACGAGACG","GGCTACATACTG","GGCTACACTAGC","GGCTACTGAGCT","GGCTACTGCTAC","GGCTACCGTACG",
             "TAGCTTTATATA","TAGCTTCGCTCT","TAGCTTGAGACG","TAGCTTATACTG","TAGCTTACTAGC","TAGCTTTGAGCT","TAGCTTTGCTAC","TAGCTTCGTACG",
             "ACGTCTTATATA","ACGTCTCGCTCT","ACGTCTGAGACG","ACGTCTATACTG","ACGTCTACTAGC","ACGTCTTGAGCT","ACGTCTTGCTAC","ACGTCTCGTACG",
            ] #concatenated multiplexing tags from the first and second reads that uniquely identify a sample, currently must have 2 or more multitags


In [7]:
directory = "/Volumes/zmliu_02/XYZ20180607/raw_data/XYZ_0424/"#directory where files are read and saved
f_gzipped_fastqfile = "XYZ_0424_USPD16098369_H23FFBBXX_L7_1.fq.gz" #The forward reads, gzip file
r_gzipped_fastqfile = "XYZ_0424_USPD16098369_H23FFBBXX_L7_2.fq.gz" #The reverse reads, gzip file
q = "fastq" #the type of fastq file coming off of the sequencer, what format are the quality scores?
f_seqtag_length = 9 #the length of the sequencing tag on the first read (UMI1)
r_seqtag_length = 9 #the length of the sequencing tag on the second read (UMI2)
f_multitag_length = 6 #the length of the multiplexing tag on the first read
r_multitag_length = 6 #the length of the multiplexing tag on the second read
f_lintag_length = 32 #the length of the lineage tag on the first read (first barcode)
r_lintag_length = 30 #the length of the lineage tag on the second read (second barcode)
f_spacer_length = 24 #distance to first barcode in forward read, not including the multitag and the seqtag
r_spacer_length = 33 #distance second barcode in reverse read, not including the multitag and the seqtag
min_qs = 30 #the minimum avareage quality score for both lineage tags
lintag_grep_filter1 ='\D*?(.CCG|A.CG|AC.G|ACC.)\D{19,21}?(.TTT|G.TT|GT.T|GTT.)\D*' #first barcode
lintag_grep_filter2 ='\D*?(.AAC|A.AC|AA.C|AAA.)\D{19,21}?(.AAA|C.AA|CA.A|CAA.)\D*' #second barcode
clip_ends = 0 #logical of whether or not to clip the front and back ends off of lintag1 and lintag2
lintag1_front_clipper = '(.CCG|A.CG|AC.G|ACC.)' #only report lintag1 after this sequence
lintag2_front_clipper = '(.AAC|A.AC|AA.C|AAA.)' #only report lintag2 after this sequence
lintag1_rear_clipper = '(.TTG|T.TG|TT.G|TTT.)' #only report lintag1 before this sequence, this must be the REVERSE of the true sequence
lintag2_rear_clipper = '(.AAC|A.AC|AA.C|AAA.)' #only report lintag2 before this sequence, this must be the REVERSE of the true sequence
#multitags = ["TGACCAATACTG", "ACAGTGTATATA", "ACAGTGCGCTCT", "ACAGTGGAGACG", "ACAGTGATACTG", "ACAGTGACTAGC", "TGACCATATATA","TGACCACGCTCT","TGACCAGAGACG","ATCACGATACTG","ATCACGACTAGC","TAGCTTTGAGCT"] #concatenated multiplexing tags from the first and second reads that uniquely identify a sample, currently must have 2 or more multitags
write_multitags = True #True will write multitags for all reads, otherwise only multitags for unmatched reads will be written
merged_output = False #True will merge double barcodes into one barcode and treat these as single barcodes
cores = 12 #the number of processors
frequency_cutoff = 1 #clusters less than this number of reads under this cutoff will not be reported by bartender

In [4]:
import BC
BC.count(directory, f_gzipped_fastqfile, r_gzipped_fastqfile,
      q = "fastq",
      f_seqtag_length = 9,
      r_seqtag_length = 9,
      f_multitag_length = 6,
      r_multitag_length = 6,
      f_lintag_length = 32,
      r_lintag_length = 30, 
      f_spacer_length = 24, #distance to first barcode in forward read (ignoring the length the thie multitag and the seqtag)
      r_spacer_length = 33, #distance second barcode in reverse read (ignoring the length the thie multitag and the seqtag)
      min_qs = 30, #the minimum avareage quality score for both lineage tags
      lintag_grep_filter1 = lintag_grep_filter1, #first barcode
      lintag_grep_filter2 = lintag_grep_filter2, #second barcode
      clip_ends = 1, #logical of whether or not to clip the front and back ends off of lintag1 and lintag2
      lintag1_front_clipper = lintag1_front_clipper, #only report lintag1 after this sequence
      lintag2_front_clipper = lintag2_front_clipper, #only report lintag2 after this sequence
      lintag1_rear_clipper = lintag1_rear_clipper, #only report lintag1 before this sequence, this must be the COMPLIMENT of the true sequence
      lintag2_rear_clipper = lintag2_rear_clipper, #only report lintag2 before this sequence, this must be the COMPLIMENT of the true sequence
      multitags = multitags, #concatenated multiplexing tags from the first and second reads that uniquely identify a sample, currently must have 2 or more multitags
      write_multitags = True, #True will write multitags for all reads, otherwise only multitags for unmatched reads will be written
      merged_output = False, #True will merge double barcodes into one barcode and treat these as single barcodes
      cores = 12, #the number of processors
      frequency_cutoff = 1) #clusters less than this number of reads under this cutoff will not be reported by bartendder

Loading XYZ_0424_USPD16098369_H23FFBBXX_L7_1.fq.gz and XYZ_0424_USPD16098369_H23FFBBXX_L7_2.fq.gz and parsing
Saving the combined forward and reverse sequencing tags as seqtag.txt
Saving the combined forward and reverse multiplexing tags  as multitag.txt
Saving the first lineage tag as lintag1.txt
Saving the first lineage tag as lintag2.txt


KeyboardInterrupt: 